In [1]:
%load_ext watermark
%watermark

2019-09-05T07:09:33-03:00

CPython 3.5.3
IPython 7.8.0

compiler   : GCC 6.3.0 20170516
system     : Linux
release    : 4.9.0-8-amd64
machine    : x86_64
processor  : 
CPU cores  : 4
interpreter: 64bit


### en este proceso Anlizamos valores duplicados, elementos vacios, y columnas que afectaran al modelo.
#### Ayer trabajamos en entender la  estructura del campo place_with_parent_names,

In [9]:
import pandas as pd
import numpy as np
from scipy import stats

# ¿Quien influye en mi modelo?
## Tipo de Propiedad
## Ubicacion:
        ### TOmamos como referencia  columna "place_with_parent_name":
        
                

In [10]:
import matplotlib.pyplot as plt
%matplotlib inline

#plt.rcParams['figure.figsize'] = (5,5)s # 10 pulgadas de ancho y 10 de alto para todos los plots( modificamos el tamaño de las graficas

In [11]:
# En el paso anterior eliminamos las columnas que no afectaran a nuestro modelo futuro; tomamos como dataSet el guardado en paso anterior
df_properati = pd.read_csv('../Data/properati_1.csv')


In [12]:
df_properati.sample()

,property_type,place_name,place_with_parent_names,state_name,geonames_id,lat-lon,lat,lon,price,currency,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
59822,house,haedo,|argentina|bs.as. g.b.a. zona oeste|morón|haedo|,bs.as. g.b.a. zona oeste,NaN,NaN,NaN,NaN,160000.0,usd,...,90.0,571.428571,1777.777778,NaN,3.0,NaN,http://www.properati.com.ar/19y03_venta_casa_h...,casa americana ideal modernizar . liv. com. co...,casa en venta haedo / moron (a195 1928),https://thumbs4.properati.com/9/ivqgkjhjls3ota...


In [13]:
df_properati['geonames_id'].value_counts().iloc[0]

8504

# VALORES  DUPLICADOS

In [14]:
#Vemos la cantidad de valores repetidos por cada columna:
def duplicados_columna(df):
        """Muestra elementos duplicados"""
        len_df = df.shape[0]
        for columna in df:
            duplicados_columna = df[columna].value_counts()
            mayor_duplic = duplicados_columna.iloc[0]
            menor_duplic = duplicados_columna.iloc[-1]
            print("{} ----{}--{}--{}".format(columna,round(mayor_duplic/(1.0 * len_df),3),round(menor_duplic/(1.0*len_df),3), df[columna].dtype))    


In [15]:
# Observamos los duplicados por cada columna
duplicados_columna(df_properati)

property_type ----0.586--0.034--object
place_name ----0.076--0.0--object
place_with_parent_names ----0.07--0.0--object
state_name ----0.267--0.0--object
geonames_id ----0.07--0.0--float64
lat-lon ----0.003--0.0--object
lat ----0.003--0.0--float64
lon ----0.003--0.0--float64
price ----0.014--0.0--float64
currency ----0.723--0.0--object
price_aprox_local_currency ----0.014--0.0--float64
price_aprox_usd ----0.014--0.0--float64
surface_total_in_m2 ----0.014--0.0--float64
surface_covered_in_m2 ----0.023--0.0--float64
price_usd_per_m2 ----0.005--0.0--float64
price_per_m2 ----0.007--0.0--float64
floor ----0.017--0.0--float64
rooms ----0.104--0.0--float64
expenses ----0.008--0.0--float64
properati_url ----0.0--0.0--object
description ----0.003--0.0--object
title ----0.041--0.0--object
image_thumbnail ----0.002--0.0--object


In [4]:
#df_properati['property_type'].value_counts(normalize=True).plot.barh()

In [31]:
#generamos un nueso DataFrame formado por :
        #Concatenars las columnas :pais, place_name, state_name para comparar para compararlo con la columna "place_with_parent_names"

df_concat ="|"+ "argentina"+"|"+df_properati['place_name']+"|"+df_properati['state_name']+"|"

In [1]:
# comparamos el nuevo dataFrame(df_concat) con la columna 'place_with_parent_names'
(df_concat == (df_properati['place_with_parent_names']))
# Observamos que hay valores  que son diferentes

In [43]:
# Observamos que el indice 6 tenian valores diferentes, procedemos a visualizar los valores ocmo tal
df_properati.iloc[6]['place_with_parent_names']

'|argentina|bs.as. g.b.a. zona norte|vicente lópez|munro|'

In [59]:
#Creamos un DataFrame cuyas columnas son los valores de place_with_parent_names, 
location_df = df_properati['place_with_parent_names'].apply(lambda x: pd.Series(x.split('|')))

In [61]:
#Observamos valores que nos genero
location_df.head()

,0,1,2,3,4,5,6
0,,argentina,capital federal,mataderos,,NaN,NaN
1,,argentina,bs.as. g.b.a. zona sur,la plata,,NaN,NaN
2,,argentina,capital federal,mataderos,,NaN,NaN
3,,argentina,capital federal,liniers,,NaN,NaN
4,,argentina,buenos aires costa atlántica,mar del plata,centro,,NaN


In [62]:
#observalos los diferentes valores  que tienen cada columna
location_df[1].value_counts() # Eliminaremos

argentina    121220
Name: 1, dtype: int64

In [2]:
location_df[2].value_counts()

In [3]:
location_df[3].value_counts()

In [4]:
location_df[5].value_counts()

In [76]:
location_df[6].value_counts()

    548
Name: 6, dtype: int64

In [78]:
#Encontramos valores repetidos en la columna 3 y 4( ejemplo:pilas | pilar), extraemos esos valores en un DataFrame,
df_col3_col4 = location_df[[3,4]]


In [81]:
#agregamos una nueva columna dif, cuyos valores son TRUE si columna3 = a columna 4(ejemplo:pilar = pilar), y FALSE di columna3!= columna4

df_col3_col4['dif'] = df_col3_col4[3] == df_col3_col4[4]
# OJO: al correr nos genera un warning, esto es propio del kernel de, no afecta a los procesos

C:\Users\marx.e.morales.tello\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [5]:
#Observamos el DF(DataFrame) generado:
df_col3_col4

In [ ]:
#Cantidad de valores falso y true q observamos:
df_col3_col4['dif'].value_counts()